In [1]:
# set correct path for imports for this notebook
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# external imports 
import logging
logging.basicConfig(level=logging.INFO)

# local imports
from src.fetch.google_reviews import fetch_google_reviews 
from src.fetch.google_place import fetch_place_id, fetch_restaurant_info, Restaurant
from src.nlp.dish_extractor import extract_dishes, _cached_extract_single
from src.ranking.functions import assign_rankings
from src.recommendation.recs import form_recommendations

# Clear dish extractor cache to ensure fresh model load
_cached_extract_single.cache_clear()


c:\Users\robin\Desktop\Bootcamp_Files\capstone-dishtip-be\backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:src.nlp.dish_extractor:✅ Using 8 CPU threads for PyTorch


In [2]:
r1_id = fetch_place_id("funkyfish berlin")
r2_id = fetch_place_id("standard pizza west berlin")
r3_id = fetch_place_id("borschert berlin")
r4_id = fetch_place_id("Gio's Berlin")

r4_id 

INFO:src.fetch.google_place:
 Found restaurant: FunkyFisch
Kantstraße 135-136, 10625 Berlin, Deutschland
ID = ChIJU7-KvONQqEcRCmPCCvo-Fg4

INFO:src.fetch.google_place:
 Found restaurant: Standard Serious Pizza Charlottenburg
Schlüterstraße 63, 10625 Berlin, Deutschland
ID = ChIJc2uwDI1RqEcRfAsy3YxyfIY

INFO:src.fetch.google_place:
 Found restaurant: Borchardt
Französische Str. 47, 10117 Berlin, Deutschland
ID = ChIJVfEmINtRqEcRsch0pslaanQ

INFO:src.fetch.google_place:
 Found restaurant: Gio's
Dresdener Str. 16, 10999 Berlin, Deutschland
ID = ChIJ65Liv5JPqEcRodr8aFFNAvE



'ChIJ65Liv5JPqEcRodr8aFFNAvE'

In [3]:
r1 = Restaurant(r1_id)
r4 = Restaurant(r4_id)

In [4]:
r1_id

'ChIJU7-KvONQqEcRCmPCCvo-Fg4'

In [5]:
r1_reviews = fetch_google_reviews(r1_id)
r2_reviews = fetch_google_reviews(r2_id)
r3_reviews = fetch_google_reviews(r3_id)
r4_reviews = fetch_google_reviews(r4_id)

INFO:src.fetch.google_reviews:✅ Retrieved 5 reviews for FunkyFisch (EN)
INFO:src.fetch.google_reviews:✅ Retrieved 5 reviews for Standard Pizza West (EN)
INFO:src.fetch.google_reviews:✅ Retrieved 5 reviews for Borchardt (EN)
INFO:src.fetch.google_reviews:✅ Retrieved 5 reviews for Gio's (EN)


In [6]:
r4_reviews

[{'id': None,
  'source': 'google',
  'author': 'Nika Chugoshvili',
  'text': 'Friendly cozy and family atmosphere with authentic food. Not too pretty but original. Taste is priority. Mini georgian bread was super nice, adjapsandali and dessert as well.',
  'dishes': [],
  'rating': 5,
  'date': 1751374444,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/109702310661589209222/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Gaia Tassinari',
  'text': 'What a wonderful dining experience. Authentic delicious food, lovely service, amazing red wine. Thank you! Will be back for more.',
  'dishes': [],
  'rating': 5,
  'date': 1755459365,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/113896279646697196969/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Steven Tice',
  'text': "First experience with Georgian food, and it was amazing! The service was fantastic, and our server was kind enoug

In [7]:
extract_dishes(r4_reviews, True)

INFO:src.nlp.dish_extractor:Loading model: google/flan-t5-large ...
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
INFO:src.nlp.dish_extractor:Model loaded successfully.
INFO:src.nlp.dish_extractor:📝 Friendly cozy and family atmosphere with authentic food. Not too pretty but original. Taste is priority. Mini georgian bread was super nice, adjapsandali and dessert as well.
INFO:src.nlp.dish_extractor:🍽️ mini georgian bread, dessert, adjapsandali

INFO:src.nlp.dish_extractor:📝 What a wonderful dining experience. Authentic delicious food, lovely service, amazing red wine. Thank you! Will be back for more.
INFO:src.nlp.dish_extractor:🍽️ none

INFO:src.nlp.dish_extractor:📝 First experience with Georgian food, and it was amazing! The service was fantastic, and our server was kind enough to help us pick out excellent options for our first Georgian food experience (especially because we

[{'id': None,
  'source': 'google',
  'author': 'Nika Chugoshvili',
  'text': 'Friendly cozy and family atmosphere with authentic food. Not too pretty but original. Taste is priority. Mini georgian bread was super nice, adjapsandali and dessert as well.',
  'dishes': [{'name': 'mini georgian bread', 'ranking': None},
   {'name': 'dessert', 'ranking': None},
   {'name': 'adjapsandali', 'ranking': None}],
  'rating': 5,
  'date': 1751374444,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/109702310661589209222/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Gaia Tassinari',
  'text': 'What a wonderful dining experience. Authentic delicious food, lovely service, amazing red wine. Thank you! Will be back for more.',
  'dishes': [],
  'rating': 5,
  'date': 1755459365,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/113896279646697196969/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Ste

In [8]:
assign_rankings(r4_reviews, True)

Assigned ranking -1 to dish 'mini georgian bread' in review #1
Assigned ranking -1 to dish 'dessert' in review #1
Assigned ranking -1 to dish 'adjapsandali' in review #1
Assigned ranking -1 to dish 'dumplings' in review #4
Assigned ranking -1 to dish 'khachapuri imeruli' in review #5
Assigned ranking -1 to dish 'khinkali with meat' in review #5
Assigned ranking -1 to dish 'khinkali with cheese' in review #5


In [9]:
r4_reviews

[{'id': None,
  'source': 'google',
  'author': 'Nika Chugoshvili',
  'text': 'Friendly cozy and family atmosphere with authentic food. Not too pretty but original. Taste is priority. Mini georgian bread was super nice, adjapsandali and dessert as well.',
  'dishes': [{'name': 'mini georgian bread', 'ranking': -1},
   {'name': 'dessert', 'ranking': -1},
   {'name': 'adjapsandali', 'ranking': -1}],
  'rating': 5,
  'date': 1751374444,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/109702310661589209222/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Gaia Tassinari',
  'text': 'What a wonderful dining experience. Authentic delicious food, lovely service, amazing red wine. Thank you! Will be back for more.',
  'dishes': [],
  'rating': 5,
  'date': 1755459365,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/113896279646697196969/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Steven Ti

In [10]:
form_recommendations(r4_reviews)


[{'dish_name': 'mini georgian bread',
  'ranking': -1,
  'author': 'Nika Chugoshvili',
  'source': 'google',
  'timestamp': 1751374444,
  'review_link': None},
 {'dish_name': 'dessert',
  'ranking': -1,
  'author': 'Nika Chugoshvili',
  'source': 'google',
  'timestamp': 1751374444,
  'review_link': None},
 {'dish_name': 'adjapsandali',
  'ranking': -1,
  'author': 'Nika Chugoshvili',
  'source': 'google',
  'timestamp': 1751374444,
  'review_link': None},
 {'dish_name': 'dumplings',
  'ranking': -1,
  'author': 'Nick Orloff',
  'source': 'google',
  'timestamp': 1759490727,
  'review_link': None},
 {'dish_name': 'khachapuri imeruli',
  'ranking': -1,
  'author': 'Angelina Fedorenko',
  'source': 'google',
  'timestamp': 1742291482,
  'review_link': None},
 {'dish_name': 'khinkali with meat',
  'ranking': -1,
  'author': 'Angelina Fedorenko',
  'source': 'google',
  'timestamp': 1742291482,
  'review_link': None},
 {'dish_name': 'khinkali with cheese',
  'ranking': -1,
  'author': 'Ang